# 🔥 FILE 1-C: torch.nn & First Neural Network

**PHẦN 1 - FOUNDATION (BEGINNER)**

---

## 📋 Nội Dung

✅ torch.nn là gì và tại sao cần thiết

✅ nn.Module - Base class cho mọi model

✅ nn.Linear - Fully connected layer

✅ Activation functions (ReLU, Sigmoid, Tanh, Softmax)

✅ nn.Sequential - Xây dựng model nhanh

✅ Custom nn.Module - Tạo model linh hoạt

✅ Loss functions (MSE, CrossEntropy)

✅ Optimizers (SGD, Adam)

✅ Dự án: MLP đơn giản

---

## 🎯 Mục Tiêu

Chuyển từ code "thủ công" sang sử dụng torch.nn để:
- Viết code ngắn gọn, dễ bảo trì
- Tự động quản lý parameters
- Xây dựng mạng neural phức tạp

---

## ⏱️ Thời Gian Học: 2-2.5 giờ

---

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")
print("✅ Ready to learn torch.nn!")

---

# 1️⃣ Tại Sao Cần torch.nn?

## So Sánh: Trước và Sau torch.nn

### Cách Cũ (FILE 1-B): Làm Thủ Công

```python
# Khởi tạo parameters
w = torch.randn(1, 1, requires_grad=True)
b = torch.zeros(1, 1, requires_grad=True)

# Định nghĩa model
def linear_model(X, w, b):
    return X @ w + b

# Training loop
for epoch in range(epochs):
    y_pred = linear_model(X, w, b)
    loss = mse_loss(y_pred, y)
    loss.backward()
    
    with torch.no_grad():
        w -= lr * w.grad
        b -= lr * b.grad
    
    w.grad.zero_()
    b.grad.zero_()
```

### Cách Mới (torch.nn): Tự Động

```python
# Tạo model
model = nn.Linear(1, 1)

# Loss và optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(epochs):
    y_pred = model(X)
    loss = criterion(y_pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
```

## Ưu Điểm của torch.nn

✅ **Ngắn gọn**: Ít code hơn nhiều

✅ **Tự động**: Quản lý parameters tự động

✅ **Linh hoạt**: Dễ xây dựng mạng phức tạp

✅ **Chuẩn**: Best practices đã được tích hợp

✅ **Dễ debug**: Cấu trúc rõ ràng

---

# 2️⃣ nn.Module - Base Class

## nn.Module Là Gì?

**nn.Module** là base class cho TẤT CẢ neural network trong PyTorch.

### Tính Năng Chính

1. ✅ Tự động quản lý parameters
2. ✅ Tự động theo dõi gradients
3. ✅ Chuyển đổi device (CPU/GPU) dễ dàng
4. ✅ Lưu/load model đơn giản
5. ✅ Training/eval mode

## Cấu Trúc nn.Module

In [ ]:
print("=" * 70)
print("CẤU TRÚC nn.Module CƠ BẢN")
print("=" * 70)

class SimpleModel(nn.Module):
    """Model đơn giản kế thừa nn.Module"""
    
    def __init__(self, input_dim, output_dim):
        super(SimpleModel, self).__init__()  # ← Bắt buộc!
        
        # Định nghĩa layers
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        """Forward pass - Định nghĩa cách tính toán"""
        return self.linear(x)

# Tạo model
model = SimpleModel(input_dim=3, output_dim=1)

print("\nModel:")
print(model)

print("\n" + "=" * 70)
print("THÔNG TIN MODEL")
print("=" * 70)

# Xem parameters
print("\nParameters:")
for name, param in model.named_parameters():
    print(f"  {name}: shape={param.shape}, requires_grad={param.requires_grad}")

# Tổng số parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTổng parameters: {total_params}")

# Test forward pass
x = torch.randn(5, 3)  # Batch=5, Features=3
y = model(x)  # Tự động gọi forward()
print(f"\nInput shape: {x.shape}")
print(f"Output shape: {y.shape}")

---

# 3️⃣ nn.Linear - Fully Connected Layer

## nn.Linear Là Gì?

**nn.Linear(in_features, out_features)** = Fully connected layer (Dense layer)

$$y = xW^T + b$$

Trong đó:
- $x$: Input (batch_size, in_features)
- $W$: Weight matrix (out_features, in_features)
- $b$: Bias vector (out_features)
- $y$: Output (batch_size, out_features)

In [ ]:
print("=" * 70)
print("nn.Linear CƠ BẢN")
print("=" * 70)

# Tạo Linear layer
linear = nn.Linear(in_features=3, out_features=2)

print(f"\nLayer: {linear}")

# Xem weight và bias
print(f"\nWeight shape: {linear.weight.shape}  ← (out_features, in_features)")
print(f"Bias shape: {linear.bias.shape}     ← (out_features,)")

print(f"\nWeight:\n{linear.weight.data}")
print(f"\nBias:\n{linear.bias.data}")

# Forward pass
x = torch.tensor([[1.0, 2.0, 3.0]])  # 1 sample, 3 features
y = linear(x)

print(f"\n" + "=" * 70)
print("FORWARD PASS")
print("=" * 70)
print(f"\nInput: {x}")
print(f"Input shape: {x.shape}")
print(f"\nOutput: {y}")
print(f"Output shape: {y.shape}")

# Tính toán thủ công để verify
print(f"\n" + "=" * 70)
print("VERIFICATION (Manual Calculation)")
print("=" * 70)
manual_output = x @ linear.weight.T + linear.bias
print(f"Manual: {manual_output}")
print(f"Match: {torch.allclose(y, manual_output)}")

In [ ]:
print("=" * 70)
print("LINEAR LAYER VỚI BATCH")
print("=" * 70)

linear = nn.Linear(4, 2)  # 4 input features → 2 output features

# Batch của 3 samples
x = torch.randn(3, 4)  # (batch_size=3, in_features=4)
y = linear(x)

print(f"\nInput shape: {x.shape}   ← (batch_size, in_features)")
print(f"Output shape: {y.shape}  ← (batch_size, out_features)")

print(f"\nInput:\n{x}")
print(f"\nOutput:\n{y}")

print("\n💡 Linear layer xử lý batch tự động!")

---

# 4️⃣ Activation Functions

## Tại Sao Cần Activation?

Nếu chỉ dùng Linear layers:
```
y = W₃(W₂(W₁x + b₁) + b₂) + b₃
  = W_total × x + b_total  ← Vẫn là linear!
```

→ Không thể học được pattern phức tạp (phi tuyến)

**Activation functions thêm tính phi tuyến!**

## Các Activation Phổ Biến

In [ ]:
print("=" * 70)
print("ACTIVATION FUNCTIONS")
print("=" * 70)

# Test với cùng input
x = torch.linspace(-3, 3, 100)

# 1. ReLU (Rectified Linear Unit)
relu = nn.ReLU()
y_relu = relu(x)

# 2. Sigmoid
sigmoid = nn.Sigmoid()
y_sigmoid = sigmoid(x)

# 3. Tanh
tanh = nn.Tanh()
y_tanh = tanh(x)

# 4. LeakyReLU
leaky_relu = nn.LeakyReLU(negative_slope=0.1)
y_leaky = leaky_relu(x)

# Visualize
plt.figure(figsize=(14, 4))

plt.subplot(1, 4, 1)
plt.plot(x.numpy(), y_relu.numpy(), 'b-', linewidth=2)
plt.title('ReLU', fontsize=12, fontweight='bold')
plt.xlabel('x')
plt.ylabel('ReLU(x)')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)

plt.subplot(1, 4, 2)
plt.plot(x.numpy(), y_sigmoid.numpy(), 'r-', linewidth=2)
plt.title('Sigmoid', fontsize=12, fontweight='bold')
plt.xlabel('x')
plt.ylabel('σ(x)')
plt.grid(True, alpha=0.3)
plt.axhline(y=0.5, color='k', linestyle='--', alpha=0.3)

plt.subplot(1, 4, 3)
plt.plot(x.numpy(), y_tanh.numpy(), 'g-', linewidth=2)
plt.title('Tanh', fontsize=12, fontweight='bold')
plt.xlabel('x')
plt.ylabel('tanh(x)')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)

plt.subplot(1, 4, 4)
plt.plot(x.numpy(), y_leaky.numpy(), 'm-', linewidth=2)
plt.title('LeakyReLU', fontsize=12, fontweight='bold')
plt.xlabel('x')
plt.ylabel('LeakyReLU(x)')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print("=" * 70)
print("ĐẶC ĐIỂM CÁC ACTIVATION")
print("=" * 70)

print("""
1. ReLU (Rectified Linear Unit)
   Formula: f(x) = max(0, x)
   Range: [0, +∞)
   ✅ Ưu: Nhanh, đơn giản, tránh vanishing gradient
   ❌ Nhược: Dying ReLU (neurons chết khi x < 0)
   💡 Dùng: Hidden layers (phổ biến nhất)

2. Sigmoid (Logistic)
   Formula: σ(x) = 1 / (1 + e^(-x))
   Range: (0, 1)
   ✅ Ưu: Smooth, output giống probability
   ❌ Nhược: Vanishing gradient, không zero-centered
   💡 Dùng: Binary classification (output layer)

3. Tanh (Hyperbolic Tangent)
   Formula: tanh(x) = (e^x - e^(-x)) / (e^x + e^(-x))
   Range: (-1, 1)
   ✅ Ưu: Zero-centered, mạnh hơn Sigmoid
   ❌ Nhược: Vẫn có vanishing gradient
   💡 Dùng: Hidden layers (RNN cũ)

4. LeakyReLU
   Formula: f(x) = max(αx, x), α=0.01
   Range: (-∞, +∞)
   ✅ Ưu: Khắc phục dying ReLU
   ❌ Nhược: Phải tune α
   💡 Dùng: Khi ReLU gặp vấn đề

""")

print("=" * 70)
print("💡 KHUYẾN NGHỊ")
print("=" * 70)
print("""
Hidden layers: ReLU (default choice)
Binary classification output: Sigmoid
Multi-class classification output: Softmax
Regression output: Không dùng activation (linear)
""")

## Softmax - Cho Multi-class Classification

In [ ]:
print("=" * 70)
print("SOFTMAX ACTIVATION")
print("=" * 70)

print("""
Softmax chuyển logits thành probabilities:

softmax(x)ᵢ = e^(xᵢ) / Σⱼ e^(xⱼ)

Đặc điểm:
  - Output: (0, 1) cho mỗi class
  - Tổng = 1
  - Dùng cho multi-class classification
""")

# Ví dụ
logits = torch.tensor([[2.0, 1.0, 0.1]])  # 3 classes
softmax = nn.Softmax(dim=1)
probs = softmax(logits)

print(f"\nLogits (raw scores): {logits}")
print(f"Probabilities: {probs}")
print(f"Sum of probs: {probs.sum().item():.4f}  ← Phải = 1")

# Predicted class
predicted_class = probs.argmax(dim=1)
print(f"\nPredicted class: {predicted_class.item()}")

# Batch example
print("\n" + "-" * 70)
print("Batch Example (3 samples, 4 classes)")
print("-" * 70)

logits_batch = torch.randn(3, 4)  # 3 samples, 4 classes
probs_batch = softmax(logits_batch)

print(f"\nLogits:\n{logits_batch}")
print(f"\nProbabilities:\n{probs_batch}")
print(f"\nSum per sample: {probs_batch.sum(dim=1)}")
print(f"Predicted classes: {probs_batch.argmax(dim=1)}")

---

# 5️⃣ nn.Sequential - Xây Dựng Model Nhanh

In [ ]:
print("=" * 70)
print("nn.Sequential - CÁCH ĐƠN GIẢN")
print("=" * 70)

# Tạo model bằng Sequential
model = nn.Sequential(
    nn.Linear(10, 64),   # Input: 10 → Hidden: 64
    nn.ReLU(),           # Activation
    nn.Linear(64, 32),   # Hidden: 64 → Hidden: 32
    nn.ReLU(),           # Activation
    nn.Linear(32, 1)     # Hidden: 32 → Output: 1
)

print("\nModel:")
print(model)

# Test forward
x = torch.randn(5, 10)  # Batch=5, Features=10
y = model(x)

print(f"\nInput shape: {x.shape}")
print(f"Output shape: {y.shape}")

# Đếm parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal parameters: {total_params:,}")

print("\n" + "=" * 70)
print("PARAMETERS BREAKDOWN")
print("=" * 70)
for i, (name, param) in enumerate(model.named_parameters()):
    print(f"{i}: {name:20s} {str(param.shape):20s} {param.numel():>8,} params")

---

# 6️⃣ Custom nn.Module - Tạo Model Linh Hoạt

In [ ]:
print("=" * 70)
print("CUSTOM nn.Module")
print("=" * 70)

class MLP(nn.Module):
    """Multi-Layer Perceptron"""
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        
        # Định nghĩa layers
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        """Forward pass"""
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Tạo model
model = MLP(input_dim=10, hidden_dim=64, output_dim=3)

print("\nModel Architecture:")
print(model)

# Test
x = torch.randn(8, 10)  # Batch=8
y = model(x)

print(f"\nInput: {x.shape}")
print(f"Output: {y.shape}")

print("\n" + "=" * 70)
print("✅ ƯU ĐIỂM CUSTOM MODULE")
print("=" * 70)
print("""
1. Linh hoạt hơn nn.Sequential
2. Có thể thêm logic phức tạp trong forward()
3. Dễ debug và customize
4. Có thể thêm helper methods
""")

---

# 7️⃣ Loss Functions

In [ ]:
print("=" * 70)
print("LOSS FUNCTIONS")
print("=" * 70)

print("""
1. nn.MSELoss() - Mean Squared Error
   Dùng cho: Regression
   Formula: MSE = mean((y_pred - y_true)²)
""")

mse_loss = nn.MSELoss()
y_pred = torch.tensor([1.0, 2.0, 3.0])
y_true = torch.tensor([1.5, 2.5, 2.8])
loss = mse_loss(y_pred, y_true)
print(f"MSE Loss: {loss.item():.4f}")

print("""
2. nn.L1Loss() - Mean Absolute Error
   Dùng cho: Regression (robust to outliers)
   Formula: MAE = mean(|y_pred - y_true|)
""")

l1_loss = nn.L1Loss()
loss = l1_loss(y_pred, y_true)
print(f"L1 Loss: {loss.item():.4f}")

print("""
3. nn.CrossEntropyLoss() - Cross Entropy
   Dùng cho: Multi-class classification
   Lưu ý: ĐÃ BAO GỒM Softmax!
""")

ce_loss = nn.CrossEntropyLoss()
# Logits (raw scores) - KHÔNG cần softmax trước!
logits = torch.tensor([[2.0, 1.0, 0.1],   # Sample 1
                       [0.5, 2.5, 0.3]])   # Sample 2
# Targets: class indices
targets = torch.tensor([0, 1])  # Class 0 và class 1
loss = ce_loss(logits, targets)
print(f"CrossEntropy Loss: {loss.item():.4f}")

print("""
4. nn.BCEWithLogitsLoss() - Binary Cross Entropy
   Dùng cho: Binary classification
   Lưu ý: ĐÃ BAO GỒM Sigmoid!
""")

bce_loss = nn.BCEWithLogitsLoss()
logits = torch.tensor([0.5, -0.3, 2.1])
targets = torch.tensor([1.0, 0.0, 1.0])
loss = bce_loss(logits, targets)
print(f"BCE Loss: {loss.item():.4f}")

---

# 8️⃣ Optimizers

In [ ]:
print("=" * 70)
print("OPTIMIZERS")
print("=" * 70)

# Tạo model mẫu
model = nn.Linear(10, 1)

print("""
1. SGD (Stochastic Gradient Descent)
   w = w - lr * gradient
   Đơn giản, ổn định
""")
sgd = torch.optim.SGD(model.parameters(), lr=0.01)
print(f"   {sgd}\n")

print("""
2. SGD with Momentum
   Thêm momentum để tăng tốc
""")
sgd_momentum = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
print(f"   {sgd_momentum}\n")

print("""
3. Adam (Adaptive Moment Estimation)
   Tự động điều chỉnh learning rate
   PHỔ BIẾN NHẤT!
""")
adam = torch.optim.Adam(model.parameters(), lr=0.001)
print(f"   {adam}\n")

print("""
4. AdamW (Adam with Weight Decay)
   Adam + L2 regularization đúng cách
""")
adamw = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
print(f"   {adamw}\n")

print("=" * 70)
print("💡 KHUYẾN NGHỊ")
print("=" * 70)
print("""
Bắt đầu: Adam (lr=0.001)
Fine-tuning: SGD with momentum
Modern: AdamW với weight decay
""")

---

# 9️⃣ DỰ ÁN: Linear Regression Với torch.nn

## So Sánh Với FILE 1-B

In [ ]:
print("=" * 70)
print("CHUẨN BỊ DỮ LIỆU")
print("=" * 70)

# Tạo dữ liệu (giống FILE 1-B)
torch.manual_seed(42)

true_w = 3.0
true_b = 2.0

n_samples = 100
X = torch.randn(n_samples, 1)
y = true_w * X + true_b + torch.randn(n_samples, 1) * 0.3

print(f"Data: {n_samples} samples")
print(f"True parameters: w={true_w}, b={true_b}")

# Visualize
plt.figure(figsize=(8, 6))
plt.scatter(X.numpy(), y.numpy(), alpha=0.5, s=30)
plt.plot(X.numpy(), (true_w * X + true_b).numpy(), 'r-', linewidth=2, 
         label=f'y = {true_w}x + {true_b}')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Linear Regression Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
print("=" * 70)
print("TRAINING VỚI torch.nn")
print("=" * 70)

# 1. Tạo model
model = nn.Linear(1, 1)

# 2. Loss và optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

print("\nModel:")
print(model)
print(f"\nInitial parameters:")
print(f"  w = {model.weight.item():.4f}")
print(f"  b = {model.bias.item():.4f}")

# 3. Training loop
n_epochs = 100
loss_history = []

print(f"\n{'Epoch':>6} {'Loss':>10} {'w':>10} {'b':>10}")
print("-" * 40)

for epoch in range(n_epochs):
    # Forward
    y_pred = model(X)
    loss = criterion(y_pred, y)
    
    # Backward
    optimizer.zero_grad()  # ← Zero gradients
    loss.backward()        # ← Compute gradients
    optimizer.step()       # ← Update parameters
    
    # Log
    loss_history.append(loss.item())
    
    if (epoch + 1) % 10 == 0:
        print(f"{epoch+1:6d} {loss.item():10.4f} {model.weight.item():10.4f} {model.bias.item():10.4f}")

print("\n" + "=" * 70)
print("✅ TRAINING HOÀN THÀNH")
print("=" * 70)
print(f"\nFinal parameters:")
print(f"  w = {model.weight.item():.4f}  (target: {true_w})")
print(f"  b = {model.bias.item():.4f}  (target: {true_b})")
print(f"  Loss: {loss.item():.4f}")

In [ ]:
# Visualize kết quả
plt.figure(figsize=(14, 5))

# Plot 1: Loss
plt.subplot(1, 2, 1)
plt.plot(loss_history, linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)

# Plot 2: Fit
plt.subplot(1, 2, 2)
with torch.no_grad():
    y_pred = model(X)

plt.scatter(X.numpy(), y.numpy(), alpha=0.5, s=30, label='Data')
plt.plot(X.numpy(), (true_w * X + true_b).numpy(), 'r-', linewidth=2, 
         label=f'True: y={true_w}x+{true_b}')
plt.plot(X.numpy(), y_pred.numpy(), 'g--', linewidth=2,
         label=f'Pred: y={model.weight.item():.2f}x+{model.bias.item():.2f}')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Result')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("SO SÁNH: FILE 1-B vs FILE 1-C")
print("=" * 70)
print("""
FILE 1-B (Thủ công):
  ❌ Nhiều code
  ❌ Phải quản lý w, b thủ công
  ❌ Update parameters phức tạp
  ✅ Hiểu rõ bên trong

FILE 1-C (torch.nn):
  ✅ Code ngắn gọn
  ✅ Tự động quản lý parameters
  ✅ Dễ mở rộng
  ✅ Best practices có sẵn
""")

---

# 🔟 DỰ ÁN: MLP Đơn Giản

## Bài Toán: Phân Loại Dữ Liệu Phi Tuyến

In [ ]:
print("=" * 70)
print("TẠO DỮ LIỆU PHI TUYẾN (Non-linear)")
print("=" * 70)

from sklearn.datasets import make_moons

# Tạo dữ liệu hình bán nguyệt
X_np, y_np = make_moons(n_samples=1000, noise=0.1, random_state=42)

# Chuyển sang tensor
X = torch.FloatTensor(X_np)
y = torch.LongTensor(y_np)

print(f"\nData shape: X={X.shape}, y={y.shape}")
print(f"Classes: {y.unique()}")

# Visualize
plt.figure(figsize=(8, 6))
plt.scatter(X[y==0, 0].numpy(), X[y==0, 1].numpy(), c='blue', label='Class 0', alpha=0.6, s=30)
plt.scatter(X[y==1, 0].numpy(), X[y==1, 1].numpy(), c='red', label='Class 1', alpha=0.6, s=30)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Non-linear Binary Classification Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 Dữ liệu này KHÔNG thể phân loại bằng linear model!")
print("   → Cần neural network với activation functions")

In [ ]:
print("=" * 70)
print("XÂY DỰNG MLP")
print("=" * 70)

class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(BinaryClassifier, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim, 2)  # 2 classes
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)  # Logits (không cần Softmax)
        return x

# Tạo model
model = BinaryClassifier(input_dim=2, hidden_dim=16)

print("\nModel:")
print(model)

total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal parameters: {total_params}")

In [ ]:
print("=" * 70)
print("TRAINING MLP")
print("=" * 70)

# Loss và optimizer
criterion = nn.CrossEntropyLoss()  # Cho classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training
n_epochs = 200
loss_history = []
acc_history = []

print(f"\n{'Epoch':>6} {'Loss':>10} {'Accuracy':>10}")
print("-" * 30)

for epoch in range(n_epochs):
    # Forward
    logits = model(X)
    loss = criterion(logits, y)
    
    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Calculate accuracy
    with torch.no_grad():
        pred = logits.argmax(dim=1)
        acc = (pred == y).float().mean()
    
    loss_history.append(loss.item())
    acc_history.append(acc.item())
    
    if (epoch + 1) % 20 == 0:
        print(f"{epoch+1:6d} {loss.item():10.4f} {acc.item():10.2%}")

print("\n✅ Training hoàn thành!")
print(f"Final accuracy: {acc_history[-1]:.2%}")

In [ ]:
# Visualize kết quả
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Loss
axes[0].plot(loss_history, linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

# Plot 2: Accuracy
axes[1].plot(acc_history, linewidth=2, color='green')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training Accuracy')
axes[1].set_ylim([0, 1])
axes[1].grid(True, alpha=0.3)

# Plot 3: Decision boundary
h = 0.01
x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

with torch.no_grad():
    Z = model(torch.FloatTensor(np.c_[xx.ravel(), yy.ravel()]))
    Z = Z.argmax(dim=1).numpy().reshape(xx.shape)

axes[2].contourf(xx, yy, Z, alpha=0.3, levels=[0, 0.5, 1], colors=['blue', 'red'])
axes[2].scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Class 0', alpha=0.6, s=20)
axes[2].scatter(X[y==1, 0], X[y==1, 1], c='red', label='Class 1', alpha=0.6, s=20)
axes[2].set_xlabel('Feature 1')
axes[2].set_ylabel('Feature 2')
axes[2].set_title('Decision Boundary')
axes[2].legend()

plt.tight_layout()
plt.show()

print("\n🎉 MLP đã học được decision boundary phi tuyến!")

---

# ✅ Tổng Kết FILE 1-C

## Bạn Đã Học Được

✅ **torch.nn** - Framework xây dựng neural networks

✅ **nn.Module** - Base class cho mọi model

✅ **nn.Linear** - Fully connected layer

✅ **Activation functions** - ReLU, Sigmoid, Tanh, Softmax

✅ **nn.Sequential** - Xây dựng model nhanh

✅ **Custom nn.Module** - Tạo model linh hoạt

✅ **Loss functions** - MSE, L1, CrossEntropy, BCE

✅ **Optimizers** - SGD, Adam, AdamW

✅ **Dự án hoàn chỉnh** - Linear Regression và MLP

---

## Training Loop Với torch.nn

```python
# Setup
model = YourModel()
criterion = nn.MSELoss()  # hoặc loss khác
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(n_epochs):
    # Forward
    y_pred = model(X)
    loss = criterion(y_pred, y)
    
    # Backward
    optimizer.zero_grad()  # Zero gradients
    loss.backward()        # Compute gradients
    optimizer.step()       # Update parameters
```

---

## Bài Tập Thực Hành

### Bài 1: Linear Regression 3D
1. Tạo data: y = 2x₁ + 3x₂ + 4x₃ + 1
2. Dùng nn.Linear(3, 1) để fit
3. So sánh parameters học được vs true

### Bài 2: Thử Các Activations
1. Tạo MLP với ReLU
2. Thay bằng Tanh
3. Thay bằng Sigmoid
4. So sánh performance

### Bài 3: Multi-class Classification
1. Tạo data 3 classes
2. Build MLP với output=3
3. Dùng CrossEntropyLoss
4. Visualize decision boundaries

### Bài 4: Deeper Network
1. Tạo MLP 5 layers
2. So sánh với 2 layers
3. Quan sát overfitting

### Bài 5: Custom Activation
1. Implement Swish: f(x) = x * sigmoid(x)
2. Dùng trong MLP
3. So sánh với ReLU

---

## Tiếp Theo

📘 **FILE 1-D: GPU, Debugging & Exercises**

Bạn sẽ học:
- CPU vs GPU
- Chuyển model lên GPU
- Common errors và cách fix
- Debugging techniques
- Tổng hợp bài tập Foundation

---